## DeepAR Model - Predict Bike Rental with Dynamic Features

Note: This dataset is not a true timeseries as there a lot of gaps

We have data only for first 20 days of each month and model needs to predict the rentals for 
the remaining days of the month. The dataset consists of two years data. DeepAR will shine with true multiple-timeseries dataset like the electricity example given below

In [38]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import datetime

import boto3
import sagemaker
from sagemaker import get_execution_role

# This code is derived from AWS SageMaker Samples:
# https://github.com/awslabs/amazon-sagemaker-examples/tree/master/introduction_to_amazon_algorithms/deepar_electricity
# https://github.com/awslabs/amazon-sagemaker-examples/tree/master/introduction_to_amazon_algorithms/deepar_synthetic

In [39]:
with_categories = False
# Set a good base job name
# It will help in identifying trained models and endpoints
base_job_name = 'deepar-AAPL-with-dynamic-feat'

In [40]:
bucket = 'mw-ml-sagemaker'
prefix = 'deepar/bikerental'

# This structure allows multiple training and test files for model development and testing
s3_data_path = "{}/{}/data_dynamic".format(bucket, prefix)
s3_output_path = "{}/{}/output".format(bucket, prefix)

In [41]:
s3_data_path,s3_output_path

('mw-ml-sagemaker/deepar/bikerental/data_dynamic',
 'mw-ml-sagemaker/deepar/bikerental/output')

In [42]:
# File name is referred as key name in S3
# Files stored in S3 are automatically replicated across
# three different availability zones in the region where the bucket was created.
# http://boto3.readthedocs.io/en/latest/guide/s3.html
def write_to_s3(filename, bucket, key):
    with open(filename,'rb') as f: # Read in binary mode
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [43]:
# Upload one or more training files and test files to S3
write_to_s3('train_dynamic_feat.json',bucket,'deepar/bikerental/data_dynamic/train/train_dynamic_feat.json')
write_to_s3('test_dynamic_feat.json',bucket,'deepar/bikerental/data_dynamic/test/test_dynamic_feat.json')

In [44]:
# Establish a session with AWS
sess = sagemaker.Session()
role = get_execution_role()

In [45]:
# This role contains the permissions needed to train, deploy models
# SageMaker Service is trusted to assume this role
print(role)

arn:aws:iam::480536818350:role/service-role/AmazonSageMaker-ExecutionRole-20201001T191047


In [46]:
# https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html#sagemaker.image_uris.retrieve

# SDK 2 uses image_uris.retrieve the container image location

# Use DeepAR Container
container = sagemaker.image_uris.retrieve("forecasting-deepar",sess.boto_region_name)

print (f'Using DeepAR Container {container}')

Using DeepAR Container 522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:1


In [47]:
container

'522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:1'

In [48]:
freq='D' # Timeseries consists Hourly Data and we need to predict hourly rental count

# how far in the future predictions can be made
# 12 days worth of hourly forecast 
prediction_length = 5

# aws recommends setting context same as prediction length as a starting point. 
# This controls how far in the past the network can see
context_length = 5

In [49]:
# Configure the training job
# Specify type and number of instances to use
#   Reference: http://sagemaker.readthedocs.io/en/latest/estimators.html
# SDK 2.x version does not require train prefix for instance count and type

estimator = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path="s3://" + s3_output_path,
    sagemaker_session=sess,
    base_job_name=base_job_name)

In [50]:
freq, context_length, prediction_length

('D', 5, 5)

In [51]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "10",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "cardinality" : "auto" if with_categories else ''
}

In [52]:
hyperparameters

{'time_freq': 'D',
 'epochs': '400',
 'early_stopping_patience': '10',
 'mini_batch_size': '64',
 'learning_rate': '5E-4',
 'context_length': '5',
 'prediction_length': '5',
 'cardinality': ''}

In [53]:
estimator.set_hyperparameters(**hyperparameters)

In [54]:
# Here, we are simply referring to train path and test path
# You can have multiple files in each path
# SageMaker will use all the files
data_channels = {
    "train": "s3://{}/train/".format(s3_data_path),
    "test": "s3://{}/test/".format(s3_data_path)
}

In [55]:
data_channels

{'train': 's3://mw-ml-sagemaker/deepar/bikerental/data_dynamic/train/',
 'test': 's3://mw-ml-sagemaker/deepar/bikerental/data_dynamic/test/'}

In [56]:
# This step takes around 35 minutes to train the model with m4.xlarge instance
estimator.fit(inputs=data_channels)

2020-12-31 22:20:03 Starting - Starting the training job...
2020-12-31 22:20:26 Starting - Launching requested ML instancesProfilerReport-1609453203: InProgress
......
2020-12-31 22:21:27 Starting - Preparing the instances for training......
2020-12-31 22:22:29 Downloading - Downloading input data
2020-12-31 22:22:29 Training - Downloading the training image...
2020-12-31 22:22:48 Training - Training image download completed. Training in progress.Arguments: train
[12/31/2020 22:22:49 INFO 140699746432832] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'num_dynamic_feat': u'auto', u'dropout_rate': u'0.10', u'mini_batch_size': u'128', u'test_quantiles': u'[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'num_eval_samples': u'100', u'learning_rate': u'0.001', u'num_cells': u'40', u'num_layers': u'2', u'embedding_dimension': u'10', u'_kvstore': u'auto', u'_num_kv_s

In [57]:
job_name = estimator.latest_training_job.name

In [58]:
print ('job name: {0}'.format(job_name))

job name: deepar-AAPL-with-dynamic-feat-2020-12-31-22-20-03-227


In [60]:
# Create an endpoint for real-time predictions
# SDK 2. parameter name for container: image_uri

endpoint_name = sess.endpoint_from_job(
    job_name=job_name,
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    image_uri=container,
    role=role)

ValueError: Endpoint with name "deepar-AAPL-with-dynamic-feat-2020-12-31-22-20-03-227" already exists; please pick a different name.

In [61]:
print ('endpoint name: {0}'.format(endpoint_name))

endpoint name: deepar-AAPL-with-dynamic-feat-2020-12-31-22-20-03-227


In [ ]:
# In the next lab, we will use the above endpoint for inference
# We will delete the endpoint in the next lab